<a href="https://colab.research.google.com/github/aimtyaem/AGO/blob/main/H2OchatbotWebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m venv enterprise --upgrade
!source enterprise/bin/activate
!python -m venv enterprise --upgrade
!source enterprise/bin/activate
!pip install h2ogpte==1.4.11
!pip install gradio torch transformers cloud-tpu-client torch_xla --upgrade

Error: Command '['/content/enterprise/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: enterprise/bin/activate: No such file or directory
Error: Command '['/content/enterprise/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: enterprise/bin/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import gradio as gr
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline

# Define the theme for the chatbot UI
theme = gr.themes.Monochrome(
    primary_hue="indigo",
    secondary_hue="blue",
    neutral_hue="slate",
    radius_size=gr.themes.sizes.radius_sm,
    font=[gr.themes.GoogleFont("Open Sans"), "ui-sans-serif", "system-ui", "sans-serif"],
)

# Get the authentication token from environment variables
auth_token = os.getenv("SECRET_TOKEN")  # Changed to properly get from environment
if not auth_token:
    raise ValueError("SECRET_TOKEN environment variable not set")

# Load the specified model and tokenizer with authentication
tokenizer = AutoTokenizer.from_pretrained(
    "h2oai/h2ogpt-gm-oasst1-en-2048-open-llama-7b-preview-700bt",
    use_auth_token=auth_token  # Added auth token
)
model = AutoModelForCausalLM.from_pretrained(
    "h2oai/h2ogpt-gm-oasst1-en-2048-open-llama-7b-preview-700bt",
    use_auth_token=auth_token  # Added auth token
)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Initialize the text generation pipeline with proper device mapping
generate_text = TextGenerationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=0 if device == "cuda" else -1  # Proper device mapping
)

# Improved generation function with better parameters
def generate(query):
    return generate_text(
        query,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
    )[0]['generated_text']

# Example queries
examples = [
    "Why is drinking water so healthy?",
    "Is there such a thing as Shallow Learning?",
    "Tell me a funny joke in German",
    "What does the 402 error mean?",
    "Can penguins fly?",
    "What's the secret to a happy life?",
    "Is it easy to train large language models?"
]

# Custom CSS to hide loading messages
css = ".generating {visibility: hidden}"

# Create the Gradio interface
with gr.Blocks(theme=theme, css=css) as demo:  # Added CSS
    gr.Markdown("""<h1><center>h2oGPT</center></h1>""")

    with gr.Row():
        with gr.Column():
            instruction = gr.Textbox(placeholder="Enter your question here", label="Question", elem_id="q-input")
            submit = gr.Button("Generate Answer", variant="primary")

    with gr.Row():
        output = gr.Markdown()

    with gr.Row():
        gr.Examples(
            examples=examples,
            inputs=[instruction],
            outputs=[output],
            fn=generate,  # Directly use generate function
            cache_examples=False
        )

    submit.click(generate, inputs=instruction, outputs=output)
    instruction.submit(generate, inputs=instruction, outputs=output)

# Launch the app without concurrency_count
demo.queue().launch(debug=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9963c1400e21ebe70e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
